In [31]:

import numpy as np
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import string
from gensim.models import Word2Vec
import joblib
import re
import gradio as gr

# Load your trained Word2Vec model
w2v_model = Word2Vec.load('/work/NLP_Project/GenreFromLyricsShared/Word2VecModels/original_w2v.model')

def preprocess(lyrics):
    #stop = stopwords.words('english')
    lyrics = re.sub('\n', ' ', lyrics)
    lyrics = re.sub('\[.*?\]', '', lyrics)
    lyrics = re.sub('\\[^\s]*', '', lyrics)
    lyrics = re.sub("'", '', lyrics)
    lyrics = lyrics.strip()
    lyrics = re.sub(r'\s+', ' ', lyrics)
    lyrics = re.sub('['+string.punctuation+']', '', lyrics)
    lyrics = lyrics.lower()
    lyrics = re.sub('chorus', '', lyrics)
    lyrics = re.sub('verse', '', lyrics)
    #lyrics = ' '.join([word for word in lyrics.split() if word not in stop])
    tokens = nltk.word_tokenize(lyrics)

    # Use Word2Vec model to generate word vectors and then calculate the mean vector
    vector = np.mean([w2v_model.wv[word] for word in tokens if word in w2v_model.wv.key_to_index], axis=0)
    return vector

def predict_genre(Artist, Title, Lyrics, File):
    # Preprocess the lyrics
    vector = preprocess(lyrics)

    # Reshape the vector to 2D array as the model.predict expects 2D array
    vector = vector.reshape(1, -1)

    # Load the trained Random Forest Classifier
    clf = joblib.load('/work/NLP_Project/GenreFromLyricsShared/random_forest.sav')

    # Get the probabilities of each class
    probabilities = clf.predict_proba(vector)[0]

    # Map each probability with the corresponding genre
    genres = clf.classes_
    result = {genre: prob for genre, prob in zip(genres, probabilities)}

    return result

description = '<img src="https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_CMYK_Green.png" alt="Spotify Logo">'

iface = gr.Interface(
    fn=predict_genre,
    inputs=[
        gr.inputs.Textbox(lines=1, placeholder='Artist Here...'),
        gr.inputs.Textbox(lines=1, placeholder='Title Here...'),
        gr.inputs.Textbox(lines=4, placeholder='Lyrics Here...'),
        #gr.inputs.File()
    ],
    outputs=gr.outputs.Label(label="Genre Suggestion"),
    description=description
)
iface.launch(debug=True, share=True)



NameError: name 'number_of_genres' is not defined

In [32]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import gradio as gr

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('/work/NLP_Project/GenreFromLyricsShared/epoch=1-step=145040.ckpt', num_labels=5)  # specify the number of genres

def preprocess(lyrics):
    return tokenizer.encode_plus(lyrics, return_tensors='pt', truncation=True, padding=True)

def predict_genre(Artist, Title, Lyrics, File):
    # Preprocess the lyrics
    inputs = preprocess(Lyrics)

    # Get the probabilities of each class
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # Map each probability with the corresponding genre
    genres = clf.classes_
    result = {genre: prob for genre, prob in zip(genres, probabilities.tolist()[0])}

    return result

description = '<img src="https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_CMYK_Green.png" alt="Spotify Logo">'

iface = gr.Interface(
    fn=predict_genre,
    inputs=[
        gr.inputs.Textbox(lines=1, placeholder='Artist Here...'),
        gr.inputs.Textbox(lines=1, placeholder='Title Here...'),
        gr.inputs.Textbox(lines=4, placeholder='Lyrics Here...'),
        #gr.inputs.File()
    ],
    outputs=gr.outputs.Label(label="Genre Suggestion"),
    description=description
)
iface.launch(debug=True, share=True)


OSError: It looks like the config file at '/work/NLP_Project/GenreFromLyricsShared/epoch=1-step=145040.ckpt' is not a valid JSON file.